# Executive Summary
**This notebook demonstrates an AI-powered multi-agent system that analyzes workout pain patterns, diagnoses probable causes, and generates evidence-based recovery plans. It combines modular agent reasoning, LLM-based inference, and a lightweight Flask interface for interactive analysis.**

User Input → Parsing Agent → Form Analysis → Injury Diagnosis → Research Agent → Prescription Plan
```mermaid
graph LR
A[User Input] --> B[ParsingAgent]
B --> C[FormAnalysisAgent]
C --> D[InjuryDiagnosisAgent]
D --> E[ResearchAgent]
E --> F[PrescriptionAgent]
F --> G[Action Plan + Output]


# Connect Google driver

In [78]:
!mkdir -p WorkoutFormChecker/knowledge_base
%cd WorkoutFormChecker/
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/WorkoutFormChecker /content/drive/MyDrive/
%cd /content/drive/MyDrive/WorkoutFormChecker


/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/WorkoutFormChecker
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/WorkoutFormChecker


In [5]:
%cd /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
!grep -n "run" master.py

/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
110:def run(user_input: str):
129:    return run(user_input)
149:    result = run(test_input)  # Uses module-level run() function


#Installation

In [ ]:
%cd /content/drive/MyDrive/WorkoutFormChecker/
!pip install -r requirements.txt

/content/drive/MyDrive/WorkoutFormChecker
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 7.9 MB/s eta 0:00:00


# Verification of Installation

In [ ]:
import fastapi
import chromadb
import transformers
import torch

print("fastapi version:", fastapi.__version__)
print("chromadb version:", chromadb.__version__)
print("transformers version:", transformers.__version__)
print("torch version:", torch.__version__)


fastapi version: 0.104.1
chromadb version: 1.2.1
transformers version: 4.35.2
torch version: 2.3.1+cu121


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# GPU or CPU?

In [ ]:
import torch

print(" Checking GPU")
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"GPU  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB" if torch.cuda.is_available() else 'No GPU')

 Checking GPU
Is CUDA supported by this system? False
CUDA version: 12.1
GPU Name: No GPU
No GPU


# Install Dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('/content/drive/MyDrive/WorkoutFormChecker/.env')

# Now you can access the variables
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
RETRIEVER_KEY = os.getenv("RETRIEVER_KEY")

client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=NVIDIA_API_KEY
)

completion = client.chat.completions.create(
  model="nvidia/llama-3.1-nemotron-nano-8b-v1",
  messages=[{"role":"user","content":"What causes Knee pain during squats"}],
  temperature=0,
  top_p=0.95,
  max_tokens=4096,
  frequency_penalty=0,
  presence_penalty=0,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")




<think>

</think>

Knee pain during squats can arise from several factors, and it's essential to identify the underlying cause for effective management. Here are common causes and potential solutions:

### **Common Causes of Knee Pain During Squats:**

1. **Knee Injury or Strain**:
   - **Meniscus Tear**: A torn meniscus (cartilage in the knee joint) can cause pain, especially during activities like squats. Symptoms include popping sensation, swelling, and instability.
   - **Ligament Tear**: Tears in ligaments (e.g., ACL, PCL) can lead to pain and instability. ACL tears are more common in women and often cause pain during squatting.
   - **Knee Cap (Meniscus) Dislocation**: A dislocated meniscus can cause pain and instability, especially when the knee is twisted or bent.

2. **Overuse or Repetitive Stress**:
   - **Knee Overuse**: Squatting with heavy weights or high volume can overuse the knee joint, leading to pain. Proper form (keeping knees aligned, not bending too deeply) can hel

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=RETRIEVER_KEY,
  base_url="https://integrate.api.nvidia.com/v1"
)

response = client.embeddings.create(
    input=["What causes Knee pain during squats"],
    model="nvidia/nv-embedqa-e5-v5",
    encoding_format="float",
    extra_body={"input_type": "query", "truncate": "NONE"}
)

print(response.data[0].embedding)


[0.01580810546875, 0.008697509765625, -0.00390625, -0.0274658203125, 0.07696533203125, 0.045684814453125, 0.00759124755859375, -0.060394287109375, 0.015045166015625, -0.059112548828125, -0.009307861328125, -0.0008234977722167969, -0.038482666015625, -0.030975341796875, 0.02789306640625, -0.01458740234375, 0.00812530517578125, -0.037750244140625, 0.0262603759765625, -0.03656005859375, 0.00719451904296875, 0.047454833984375, 0.003345489501953125, 0.04150390625, -0.034454345703125, -0.031707763671875, 0.04644775390625, 0.03887939453125, -0.0025920867919921875, 0.0291290283203125, -0.042999267578125, -0.005077362060546875, -0.044586181640625, -0.04254150390625, -0.002918243408203125, -0.032867431640625, 0.00786590576171875, 0.012481689453125, -0.00848388671875, 0.037445068359375, -0.033599853515625, 0.0157318115234375, -0.0106964111328125, -0.00037860870361328125, -0.06610107421875, 0.057159423828125, 0.0219268798828125, -0.0162353515625, 0.047332763671875, -0.00922393798828125, 0.00796508

# Test the Agents Individually

In [ ]:
# Test AgentStep
class AgentStep:
    def __init__(self, step_number, action, reasoning, retrieved_docs=[], llm_output=""):
        self.step_number = step_number
        self.action = action
        self.reasoning = reasoning
        self.retrieved_docs = retrieved_docs
        self.llm_output = llm_output

# Test AgentState
class AgentState:
    def __init__(self):
        self.steps = []
        self.current_step = 0
        self.max_iterations = 7

    def add_step(self, action, reasoning):
        self.current_step += 1
        step = AgentStep(self.current_step, action, reasoning)
        self.steps.append(step)

# Test it!
state = AgentState()
state.add_step("TEST_ACTION", "Testing the agent state")
print(f"✅ Steps: {len(state.steps)}")
print(f"✅ Current step: {state.current_step}")

✅ Steps: 1
✅ Current step: 1


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# ============ CONFIGURATION ============

load_dotenv('/content/drive/MyDrive/WorkoutFormChecker/.env')
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
RETRIEVER_KEY = os.getenv("RETRIEVER_KEY")

# ============ LLM HELPER ============

def call_llm(prompt, max_tokens=1024):
    """Call Nemotron LLM"""
    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",
        api_key=NVIDIA_API_KEY
    )

    completion = client.chat.completions.create(
    model="nvidia/llama-3.1-nemotron-nano-8b-v1",
    messages=[{"role":"user","content":"What causes Knee pain during squats"}],
    temperature=0,
    top_p=0.95,
    max_tokens=4096,
    frequency_penalty=0,
    presence_penalty=0,
    stream=True
    )

    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        return chunk.choices[0].delta.content
# ============ RETRIEVER FUNCTIONS ============

def get_embedding(text):
    """Get embeddings from NVIDIA API"""

    client = OpenAI(
        base_url="https://integrate.api.nvidia.com/v1",  # Fixed: base_url not url, added v1
        api_key=RETRIEVER_KEY
    )

    response = client.embeddings.create(
        input=[text],  # Fixed: use the text parameter, not hardcoded
        model="nvidia/nv-embedqa-e5-v5",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )  # Fixed: ) not }

    return response.data[0].embedding

def retrieve_documents(query, collection_name, top_k=3):
    """Retrieve relevant documents from ChromaDB"""
    # This will be implemented after we set up ChromaDB
    # For now, return empty list
    return []

print("Retriever functions added!")
# ============ DATA MODELS ============

class AgentStep:
    """Represents one step in the agent's reasoning chain"""

    def __init__(self, step_number, action, reasoning, retrieved_docs=[], llm_output=""):
        self.step_number = step_number
        self.action = action
        self.reasoning = reasoning
        self.retrieved_docs = retrieved_docs
        self.llm_output = llm_output

    def to_dict(self):
        """Convert to dictionary for JSON serialization"""
        return {
            "step_number": self.step_number,
            "action": self.action,
            "reasoning": self.reasoning,
            "retrieved_docs": self.retrieved_docs,
            "llm_output": self.llm_output
        }

print("AgentStep class added!")

# ============ AGENT STATE ============

class AgentState:
    """Tracks agent's current state and decisions"""

    def __init__(self):
        self.steps = []
        self.current_step = 0
        self.max_iterations = 7
        self.extracted_info = {}
        self.retrieved_context = {}
        self.root_cause = None
        self.is_done = False

    def add_step(self, action, reasoning, retrieved_docs=[], llm_output=""):
        """Add a step to the reasoning chain"""
        self.current_step += 1
        step = AgentStep(
            step_number=self.current_step,
            action=action,
            reasoning=reasoning,
            retrieved_docs=retrieved_docs,
            llm_output=llm_output
        )
        self.steps.append(step)
        return step

    def should_continue(self):
        """Check if agent should keep reasoning"""
        return not self.is_done and self.current_step < self.max_iterations

print("AgentState class added!")

if __name__ == "__main__":
  print(" Agent module loaded!")

Retriever functions added!
AgentStep class added!
AgentState class added!
 Agent module loaded!


In [ ]:
# ============ AGENT ORCHESTRATOR ============

class WorkoutAgent:
    """Main agentic orchestrator - makes decisions and coordinates everything"""

    def __init__(self):
        self.state = None

    def run(self, user_input):
        """Main agentic loop"""
        # Initialize state
        self.state = AgentState()

        # Step 1: Parse user input
        self._parse_input(user_input)

        # Step 2: Agentic reasoning loop
        while self.state.should_continue():
            next_action = self._decide_next_action()

            if next_action == "RETRIEVE_FORM":
                self._retrieve_form_guides()
            elif next_action == "RETRIEVE_INJURY":
                self._retrieve_injury_patterns()
            elif next_action == "REASON":
                self._reason_root_cause()
            elif next_action == "RETRIEVE_CORRECTIVE":
                self._retrieve_correctives()
            elif next_action == "GENERATE":
                final_answer = self._generate_plan()
                self.state.is_done = True
                return final_answer, self.state.steps

        return "Could not complete analysis", self.state.steps

    def _parse_input(self, user_input):
        """Parse user input to extract key info"""
        prompt = f"""Extract key information from this workout issue:

User Input: {user_input}

Extract and respond ONLY with JSON format:
{{
    "exercise": "name of exercise",
    "pain_location": "where it hurts",
    "pain_timing": "when during movement"
}}"""

        response = call_llm(prompt, max_tokens=200)

        # Try to parse JSON
        import json
        try:
            self.state.extracted_info = json.loads(response)
        except:
            self.state.extracted_info = {"exercise": "unknown"}

        self.state.add_step(
            "PARSE_INPUT",
            "Extracted key information from user input",
            llm_output=response
        )

    def _decide_next_action(self):
        """Agent decides what to do next"""
        if "form_guides" not in self.state.retrieved_context:
            return "RETRIEVE_FORM"
        elif "injury_patterns" not in self.state.retrieved_context:
            return "RETRIEVE_INJURY"
        elif not self.state.root_cause:
            return "REASON"
        elif "correctives" not in self.state.retrieved_context:
            return "RETRIEVE_CORRECTIVE"
        else:
            return "GENERATE"

    def _retrieve_form_guides(self):
        """Retrieve exercise form guides"""
        exercise = self.state.extracted_info.get("exercise", "squat")
        docs = retrieve_documents(f"{exercise} proper form", "form_guides", top_k=3)

        self.state.retrieved_context["form_guides"] = docs
        self.state.add_step(
            "RETRIEVE_FORM",
            f"Retrieved form guides for {exercise}",
            retrieved_docs=docs
        )

    def _retrieve_injury_patterns(self):
        """Retrieve similar injury patterns"""
        pain = self.state.extracted_info.get("pain_location", "")
        docs = retrieve_documents(f"injury {pain}", "injury_patterns", top_k=3)

        self.state.retrieved_context["injury_patterns"] = docs
        self.state.add_step(
            "RETRIEVE_INJURY",
            "Retrieved similar injury patterns",
            retrieved_docs=docs
        )

    def _reason_root_cause(self):
        """Reason about root cause"""
        form_guides = "\n".join(self.state.retrieved_context.get("form_guides", []))
        injury_patterns = "\n".join(self.state.retrieved_context.get("injury_patterns", []))

        prompt = f"""Analyze this workout issue and determine ROOT CAUSE:

Exercise: {self.state.extracted_info.get('exercise')}
Pain: {self.state.extracted_info.get('pain_location')} during {self.state.extracted_info.get('pain_timing')}

Form Guidelines:
{form_guides if form_guides else "No form guides available"}

Injury Patterns:
{injury_patterns if injury_patterns else "No injury patterns available"}

Determine the ROOT CAUSE in 1-2 sentences."""

        response = call_llm(prompt, max_tokens=300)
        self.state.root_cause = response

        self.state.add_step(
            "REASON",
            "Analyzed and determined root cause",
            llm_output=response
        )

    def _retrieve_correctives(self):
        """Retrieve corrective exercises"""
        docs = retrieve_documents(f"corrective exercises {self.state.root_cause[:100]}", "correctives", top_k=3)

        self.state.retrieved_context["correctives"] = docs
        self.state.add_step(
            "RETRIEVE_CORRECTIVE",
            "Retrieved corrective exercises",
            retrieved_docs=docs
        )

    def _generate_plan(self):
        """Generate final action plan"""
        correctives = "\n".join(self.state.retrieved_context.get("correctives", []))

        prompt = f"""Create an action plan for this workout issue:

ROOT CAUSE: {self.state.root_cause}

Corrective Exercises:
{correctives if correctives else "General mobility work"}

Create a plan with:
1. IMMEDIATE ACTION (next workout)
2. THIS WEEK (daily work)
3. MONITOR (what to track)
4. SEE A PRO IF (warning signs)

Be specific and actionable."""

        response = call_llm(prompt, max_tokens=800)

        self.state.add_step(
            "GENERATE",
            "Generated personalized action plan",
            llm_output=response
        )

        return response

print("WorkoutAgent class created!")

# TEST IT
agent = WorkoutAgent()
print("Agent instantiated successfully!")

WorkoutAgent class created!
Agent instantiated successfully!


In [ ]:
# Test the complete agent
from importlib import reload
import sys

# Reload the agent module to get latest changes
if 'agent' in sys.modules:
    del sys.modules['agent']

sys.path.append('/content/drive/MyDrive/WorkoutFormChecker')
import agent

# Create agent instance
workout_agent = agent.WorkoutAgent()

# Test with a real query
print("🧪 Testing Agent...\n")

result, steps = workout_agent.run("My shoulder hurts during bench press")

print("\n" + "="*50)
print("📊 FINAL RESULT:")
print("="*50)
print(result)

print("\n" + "="*50)
print(f"🔍 REASONING STEPS: {len(steps)} steps")
print("="*50)
for step in steps:
    print(f"\nStep {step.step_number}: {step.action}")
    print(f"Reasoning: {step.reasoning}")

ModuleNotFoundError: No module named 'agent'

In [ ]:
# Find and replace the prompt in _generate_plan
with open('/content/drive/MyDrive/WorkoutFormChecker/agent.py', 'r') as f:
    content = f.read()

# Replace the old prompt
old_prompt = '''prompt = f"""Create an action plan for this workout issue:

ROOT CAUSE: {self.state.root_cause}

Corrective Exercises:
{correctives if correctives else "General mobility work"}

Create a plan with:
1. IMMEDIATE ACTION (next workout)
2. THIS WEEK (daily work)
3. MONITOR (what to track)
4. SEE A PRO IF (warning signs)

Be specific and actionable."""'''

new_prompt = '''prompt = f"""You are an expert strength coach. Create a CONCISE, SPECIFIC action plan.

ROOT CAUSE: {self.state.root_cause}

Corrective Exercises:
{correctives if correctives else "General mobility work"}

Format EXACTLY like this:

🎯 ROOT CAUSE
[1-2 sentences explaining the issue]

⚡ IMMEDIATE ACTION (Next Workout)
- [Specific change 1]
- [Specific change 2]
- [Specific change 3]

🔧 THIS WEEK (Daily Work)
- [Exercise 1: 3x15 reps]
- [Exercise 2: 10 minutes daily]
- [Exercise 3: Before each workout]

📊 MONITOR
- [Track this metric]
- [Reassess in X days]

⚠️ SEE A PRO IF
- [Red flag 1]
- [Red flag 2]

Keep under 300 words. Be SPECIFIC, not generic. No "Additional Tips" section."""'''

content = content.replace(old_prompt, new_prompt)

with open('/content/drive/MyDrive/WorkoutFormChecker/agent.py', 'w') as f:
    f.write(content)

print("✅ Improved the output format!")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WorkoutFormChecker/agent.py'

In [79]:
from IPython.display import HTML

# Force reload by reading the file again
with open('/content/drive/MyDrive/WorkoutFormChecker/frontend.html', 'r') as f:
    html_content = f.read()

HTML(html_content)

In [ ]:
%cd /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
!python master.py

/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
BaseAgent class with log_action created!
 Testing Master Orchestrator...

 Initializing Master Orchestrator...
 All agents initialized!

MULTI-AGENT ANALYSIS STARTED

 STEP 1: Parsing user input...
 Extracted: squats | right knee | after workout

 STEP 2: Analyzing exercise form...
 Form analysis complete (confidence: high)

 STEP 3: Diagnosing injury pattern...
 Diagnosis complete (confidence: high)

 STEP 4: Researching supporting evidence...
 Searching knowledge base for: squats right knee form correction
 Searching web for: squats right knee injury treatment 2024
 Found 1 web sources

 STEP 5: Creating personalized action plan...
 Action plan generated!

 MULTI-AGENT ANALYSIS COMPLETE

 FINAL RESULTS

ACTION PLAN:
**ROOT CAUSE**
Patellar Tracking Deviation caused by right knee misalignment, compressing lateral meniscus.

**IMMEDIATE ACTION (Next Workout)**
1. **Squat with knee alignment focus**: Assume a wider stance and eng

In [ ]:
!zip -r my_project.zip /content/drive/MyDrive/WorkoutFormChecker/

  adding: content/drive/MyDrive/WorkoutFormChecker/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/excercise_forms.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/injury_patterns.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/corrective_excercises.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/load_kb.ipynb (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/README.md (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.gitignore (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/requirements.txt (deflated 32%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.env (deflated 14%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/__pycache__/ (stored 0%)
  add

In [ ]:
from google.colab import files
files.download("WorkoutFormChecker.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Check if server.py exists
!ls -la /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py

# Check if frontend.html exists (might be in different location)
!ls -la /content/drive/MyDrive/WorkoutFormChecker/frontend.html

-rw------- 1 root root 6634 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py
-rw------- 1 root root 13717 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/frontend.html


In [ ]:
!ls -la /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py

-rw------- 1 root root 6634 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py


##Testing of FLASK APP


In [ ]:
from threading import Thread
import time

def run_flask():
    import os
    os.chdir('/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra')
    from server import app  # or however your app is defined
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in background
thread = Thread(target=run_flask)
thread.start()

# Give it time to start
time.sleep(3)
print("✅ Flask server should be running on port 5000")

 * Serving Flask app 'server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


✅ Flask server should be running on port 5000


In [13]:
from pyngrok import ngrok
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/WorkoutFormChecker/.env')
NGROK_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_TOKEN)



# Server Connection Issue
## -> Kill the server.
## -> Connect to 8081.
## -> Expose the Base URl.

In [75]:
!pkill -f "server.py" || true
!pkill -f "flask" || true
!pkill -f "http.server"
from pyngrok import ngrok
ngrok.kill()



^C
^C


In [76]:
!nohup python3 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py > server.log 2>&1 &
!sleep 3
!lsof -i :5000


COMMAND   PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
python3 51758 root    5u  IPv4 1619026      0t0  TCP *:5000 (LISTEN)
python3 51768 root    5u  IPv4 1619026      0t0  TCP *:5000 (LISTEN)
python3 51768 root    6u  IPv4 1619026      0t0  TCP *:5000 (LISTEN)


In [77]:
from pyngrok import ngrok
backend_url = ngrok.connect(5000)
print("Backend URL:", backend_url.public_url)


Backend URL: https://rebeca-groutiest-incorporeally.ngrok-free.dev


### Conclusion
**This notebook demonstrates how structured reasoning and modular agent design can automate workout injury diagnostics. Future versions will integrate pose estimation and personalized recovery tracking to bridge the gap between AI insights and real-world physical therapy.**
